In [1]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
#pip install SQLAlchemy

In [3]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

In [4]:
# Load the data into a DataFrame

data = pd.read_excel('data.xlsx',sheet_name='masterdata')
data1 = pd.read_excel('data.xlsx',sheet_name='price')
data2 = pd.read_excel('data.xlsx',sheet_name='Mat_attributes')
df = pd.DataFrame(data)
df1= pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18093 entries, 0 to 18092
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Product No           18093 non-null  object
 1   Prod Categoty        18093 non-null  object
 2   Product Description  18093 non-null  object
dtypes: object(3)
memory usage: 424.2+ KB


In [16]:
cols=[]

for i in df.columns:
    i=i.strip().split(' ')
    i=('_').join(i)
    cols.append(i)
df.columns = cols

In [17]:
cols=[]

for i in df1.columns:
    i=i.strip().split(' ')
    i=('_').join(i)
    cols.append(i)
df1.columns = cols

In [18]:
cols=[]

for i in df2.columns:
    i=i.strip().split(' ')
    i=('_').join(i)
    cols.append(i)
df2.columns = cols

In [19]:
# Replace these variables with your actual SQL Server credentials and database information
server = 'XXXXX'
database = 'XXX'
username = 'XX
password = 'XXXX'
driver = '{ODBC Driver 17 for SQL Server}'  # Driver version may vary, depending on your setup

In [20]:
#Establish the connection
try:
    conn = pyodbc.connect(
        f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    )
    cursor = conn.cursor()
    print("Connected to SQL Server")
except pyodbc.Error as e:
    print(f"Error connecting to SQL Server: {e}")
    exit(1)

Connected to SQL Server


In [26]:
df1.columns


Index(['M_no', 'V_no', 'Pl_Name', 'Pl_Country', 'Price'], dtype='object')

In [27]:
# Define the table name for your new table
table_name = 'masterdata'
primarykey = 'V_No'

table_name1 = 'priceinformation'
primarykey1 = 'M_no'

table_name2 = 'attributemamaster'
primarykey2 = 'M_no'

In [ ]:

# Generate the CREATE TABLE statement based on DataFrame schema
create_table_sql = f"CREATE TABLE {table_name} ("

# Data type mapping dictionary
type_mapping = {
    'int64': 'INT',
    'float64': 'FLOAT',
    'object': 'VARCHAR(255)',  # Adjust the max length as needed
}

for column, dtype in df.dtypes.items():
    sql_type = type_mapping.get(str(dtype), 'varchar(255)')  # Default to VARCHAR if data type not in the mapping
    print(sql_type)
    if column == primarykey :
        create_table_sql += f"\n{column} {sql_type} PRIMARY KEY,"
    else:
        create_table_sql += f"\n{column} {sql_type} NULL,"

In [28]:
# Generate the CREATE TABLE statement based on DataFrame schema (priceinformation)
create_table_sql1 = f"CREATE TABLE {table_name1} ("

# Data type mapping dictionary
type_mapping = {
    'int64': 'INT',
    'float64': 'FLOAT',
    'object': 'VARCHAR(255)',  # Adjust the max length as needed
}

for column, dtype in df1.dtypes.items():
    sql_type = type_mapping.get(str(dtype), 'varchar(255)')  # Default to VARCHAR if data type not in the mapping
    print(sql_type)
    if column == primarykey :
        create_table_sql1 += f"\n{column} {sql_type} PRIMARY KEY,"
    else:
        create_table_sql1 += f"\n{column} {sql_type} NULL,"

VARCHAR(255)
INT
VARCHAR(255)
VARCHAR(255)
FLOAT


In [29]:
# Generate the CREATE TABLE statement based on DataFrame schema (attributemaster)
create_table_sql2 = f"CREATE TABLE {table_name2} ("

# Data type mapping dictionary
type_mapping = {
    'int64': 'INT',
    'float64': 'FLOAT',
    'object': 'VARCHAR(255)',  # Adjust the max length as needed
}

for column, dtype in df2.dtypes.items():
    sql_type = type_mapping.get(str(dtype), 'varchar(255)')  # Default to VARCHAR if data type not in the mapping
    print(sql_type)
    if column == primarykey :
        create_table_sql2 += f"\n{column} {sql_type} PRIMARY KEY,"
    else:
        create_table_sql2 += f"\n{column} {sql_type} NULL,"

VARCHAR(255)
VARCHAR(255)
VARCHAR(255)


In [30]:
print(create_table_sql1)

CREATE TABLE priceinformation (
M_NO VARCHAR(255) NULL,
Vendor_no INT NULL,
Plant_Name VARCHAR(255) NULL,
Plant_Country VARCHAR(255) NULL,
Net_Price FLOAT NULL,


In [31]:
print(create_table_sql2)

CREATE TABLE attributemaster (
M_NO VARCHAR(255) NULL,
Commodity_Level_2 VARCHAR(255) NULL,
Product_Description VARCHAR(255) NULL,


In [ ]:
# Remove the trailing comma and close the statement
create_table_sql = create_table_sql.rstrip(",") + "\n);"

In [32]:
create_table_sql1 = create_table_sql1.rstrip(",") + "\n);"

In [33]:
create_table_sql2 = create_table_sql2.rstrip(",") + "\n);"

In [ ]:
# Execute the CREATE TABLE statement
query = create_table_sql
try:
    cursor.execute(query)
    conn.commit()
    print(f"Table '{table_name}' created successfully.")
except pyodbc.Error as e:
    print(f"Error creating table: {e}")
    conn.rollback()

In [34]:
query1 = create_table_sql1
try:
    cursor.execute(query1)
    conn.commit()
    print(f"Table '{table_name1}' created successfully.")
except pyodbc.Error as e:
    print(f"Error creating table: {e}")
    conn.rollback()

Table 'priceinformation' created successfully.


In [35]:
query2 = create_table_sql2
try:
    cursor.execute(query2)
    conn.commit()
    print(f"Table '{table_name2}' created successfully.")
except pyodbc.Error as e:
    print(f"Error creating table: {e}")
    conn.rollback()

Table 'attributemaster' created successfully.


In [ ]:
print(cursor)

In [ ]:
#insert to table
for _, row in df.iterrows():
    row = [None if pd.isna(value) else value for value in row]
    insert_data_sql = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({', '.join(['?'] * len(df.columns))})"
    cursor.execute(insert_data_sql, tuple(row))

In [36]:
for _, row in df1.iterrows():
    row = [None if pd.isna(value) else value for value in row]
    insert_data_sql1 = f"INSERT INTO {table_name1} ({', '.join(df1.columns)}) VALUES ({', '.join(['?'] * len(df1.columns))})"
    cursor.execute(insert_data_sql1, tuple(row))

In [37]:
for _, row in df2.iterrows():
    row = [None if pd.isna(value) else value for value in row]
    insert_data_sql2 = f"INSERT INTO {table_name2} ({', '.join(df2.columns)}) VALUES ({', '.join(['?'] * len(df2.columns))})"
    cursor.execute(insert_data_sql2, tuple(row))

In [38]:
conn.commit()